In [1]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

import os
import re
import pandas as pd

from IPython.display import clear_output
from IPython.display import Markdown, display

spacy.prefer_gpu()

c:\users\justin\appdata\local\programs\python\python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'
c:\users\justin\appdata\local\programs\python\python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'


False

In [2]:
nlp = spacy.load("nl_core_news_lg")

In [3]:
df = pd.read_csv('..\\data\\ocred\\files_df.csv', index_col = 0)

In [13]:
def printHilight(string):
    print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')

def printHilightUnderline(string):
    print('\033[4m'+string + ' ' +'\x1b[0m', end='')
    
def showMatches(doc, tagList):   
    indexOfMatches = []
    for ent in doc.ents:
        for i in range(int(ent.start), int(ent.end)):
            indexOfMatches.append(i)

    indexOfMatches = set(indexOfMatches)

    for token in doc:
        flag = False
        for tag in tagList:
            if tag in token.text.lower():
                flag = True
                break
        
        if token.i in indexOfMatches:
            printHilight(str(token.text))
            
        elif flag:
            printHilightUnderline(str(token.text))
            
        else:
            print(token, end=' ')
    
    return

def showEnts(doc, tagList):   
    indexOfMatches = []
    for ent in doc.ents:
        for i in range(int(ent.start), int(ent.end)):
            indexOfMatches.append(i)

    indexOfMatches = set(indexOfMatches)

    for token in doc:
        flag = False
        for tag in tagList:
            if tag in token.text.lower():
                flag = True
                break
        
        if token.i in indexOfMatches:
            printHilight(str(token.text))
            
        elif flag:
            printHilightUnderline(str(token.text))
            
        else:
            print(token, end=' ')
    return


def showMatches(doc, matches, tagList):   
    indexOfMatches = []
    for matchid, start, end in matches:
        for i in range(start, end):
            indexOfMatches.append(i)
            
    indexOfMatches = set(indexOfMatches)
    
    for token in doc:
        flag = False
        for tag in tagList:
            if tag in token.text.lower():
                flag = True
                break
        
        if token.i in indexOfMatches:
            printHilight(str(token.text))
            
        elif flag:
            printHilightUnderline(str(token.text))
            
        else:
            print(token, end=' ')

    return

In [5]:
def tagTest():
    tags = ['artikel', 'artikelen', 'artikels', 'lid', 'grondwet', 'wetten', 'art.']

    while True:
        flag = False
        test = df.sample(1).text.values[0]

        for tag in tags:
            if tag in str(test):
                flag = True
                break
        if flag:
            break
                
    
    showEnts(nlp(test), tags)
            
tagTest()

ik dus . Als ik u goed begrijp , zegt u eigenlijk : wij vinden dat 
 je per maatregel een soort wetstraject in zou moeten gaan . 
 Mijn vraag aan de heer Hijink is dan of dat niet gewoon 
 veel te lang gaat duren . Ik snap vanuit een soort staatsrech- 
 telijk oogpunt wel dat dat misschien de meest koninklijke 
 route is , maar ziet u ook dat we dan bij elke maatregel 
 weken aan de gang zijn ? 

 De heer Hijink ( SP ): 

 Volgens mij is het eerder omgekeerd . We zien juist dat het 
 in ieder geval ook niet heeft gewerkt door het op zo'n 
 rommelige en in beginsel heel chaotische manier te doen 
 met een slechte wet . Wij hadden het veel beter gevonden 
 — ik snap ook wel dat dat nu niet meer kan , omdat we dat 
 punt al voorbij zijn — als het kabinet in het begin , in mei of 
 juni , met veel beperktere wetten was gekomen die we hier 
 in alle openheid hadden kunnen bespreken , zonder allerlei 
 machtigingen voor de minister . Dan hadden we veel eerder 
 een goede grondslag gehad voor

In [22]:
ordinals = ['eerste','tweede','derde','vierde','vijfde','zesde','zevende','achtste','negende','tiende']

basePattern = [{"LOWER" : {"IN" : ['artikel', 'artikelen']}},
            {"IS_ASCII" : True},
            {"IS_PUNCT" : True, "OP" : "?", "TEXT":','},
            {"LOWER" : {"IN" : ordinals}},
            {"LOWER" : "lid"}]

basePatternExtended = [{"LOWER" : {"IN" : ['artikel', 'artikelen']}},
            {"IS_ASCII" : True},
            {"IS_PUNCT" : True, "OP" : "?", "TEXT":','},
            {"LOWER" : {"IN" : ordinals}, "OP" : "?"},
            {"LOWER" : "lid", "OP" : "?"},
            {"LOWER" : "van"},
            {"LOWER" : {"IN" : ["de", "het"]}},
            {"IS_ASCII" : True},
            {"IS_ASCII" : True, "OP" : "?"},
            {"POS" : "PROPN", "OP" : "?"}]

In [18]:
def test(text, nlp, pattern):
    tags = ['artikel', 'artikelen', 'artikels', 'lid', 'grondwet', 'wetten', 'art.']
    matcher = Matcher(nlp.vocab)
    matcher = Matcher(nlp.vocab)
    matcher.add("tests", [pattern])
    doc = nlp(text)
    matches = matcher(doc)
    showMatches(doc, matches, tags)

In [23]:
def patterns(nlp, basePattern, basePatternExtended):
    tags = ['artikel', 'artikelen', 'artikels', 'lid', 'grondwet', 'wetten', 'art.']

    while True:        
        flag = False
        text = df.sample(1).text.values[0]

        for tag in tags:
            if tag in str(text):
                flag = True
                break
                
        if not flag:
            continue
            
        matcher = Matcher(nlp.vocab)
        matcher.add("Artikel", [basePattern])
        matcher.add("Artikel extended", [basePatternExtended])
        doc = nlp(text)
        matches = matcher(doc)
        print(matches)
        showMatches(doc, matches, tags)
        if input() == 'q':
            return
        clear_output()
    
    
            
patterns(nlp, basePattern, basePatternExtended)

[(10414211751966827452, 526, 531), (10414211751966827452, 526, 532), (16945188806639740781, 560, 565)]
32480011 

 vaststellen van ministeriële regelingen . Wat betreft de toepassing van 
 hoofdstuk Va op zeeschepen en hun bemanning , geldt dat de Nederlandse 
 regelgeving in principe van toepassing is op alle schepen binnen het 
 Nederlandse territorium . Voor schepen onder buitenlandse vlag geldt het 
 uitgangspunt van het VN-Zeerechtverdrag dat de staat onder wiens vlag 
 een schip vaart , rechtsmacht heeft over dat schip . Dit volgt onder meer uit 
 de artikelen 87 , 90 en 92 van het Zeerechtverdrag . In beginsel gaat 
 Nederland daarmee niet over de interne bedrijfsvoering op een buiten- 
 lands schip en dient het terughoudend te zijn met het toepassen van 
 nationale regelgeving . Indien een ( buitenlandse ) vlagstaat andere of 
 stengere eisen stelt , kan het zijn dat personen de Nederlandse haven wel 
 in mogen , maar niet aan boord worden toegelaten . 

 De Wpg is niet van toe

# current results

Found patterns:
- "artikel 1, eerste lid"

Partial finds
- "artikel 1, eerte lid van de Modelnoodverordening"
- "artikel 39 van de Wet veiligheidsregio"
- "Artikel 151e-151i van het Wetboek van Strafvordering"

Not found patterns:
- "artikel 1", easy fix
- "eerste lid", easy fix



Names of laws aren't extracted very well by spacy which is why this is is done on top of spacy.
"Wet op de economische delicten" is recognised as just "Wet op de"

One idea of knowing what law is used when the text states "artikel 1" is to check what the previous mentioned law is. Problem is that spacy is bad at finding these 